In [10]:
import pandas as pd 
import numpy as np
import tensorflow as tf
import pickle

In [11]:
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [12]:
# train=pd.read_pickle("VDISC_train1.pickle")
# # validate=pd.read_pickle("VDISC_validate.pickle")
# test=pd.read_pickle("VDISC_test1.pickle")

# for dataset in [train,test]:
#     for col in range(1,6):
#         dataset.iloc[:,col] = dataset.iloc[:,col].map({False: 0, True: 1})

# # Create source code sdata for tokenization
# x_all = train['functionSource']
# x_all = [x.decode('utf-8') for x in x_all]
# #x_all = x_all.append(validate['functionSource'])
# #x_all = x_all.append(test['functionSource'])

# # Tokenizer with word-level
# tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=False)
# tokenizer.fit_on_texts(list(x_all))
# del(x_all)

In [13]:
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer= pickle.load(handle)

In [14]:
tokenizer

In [15]:
model = tf.keras.models.load_model("vulcnn.h5")


In [16]:
# texts=tokenizer.sequences_to_texts(list_tokenized_text)

reverse_word_map= dict(map(reversed, tokenizer.word_index.items()))

def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)

 Creating texts 
my_texts = list(map(sequence_to_text, list_tokenized_text))

In [17]:
source_code ="static void goodG2B2() size_t data ; data = 0;if ( STATIC_CONST_FIVE == 5 )data = 20;if ( STATIC_CONST_FIVE == 5 )char * myString ;if ( data > strlen ( HELLO_STRING ) )myString = ( char * ) malloc ( data * sizeof ( char ) ); strcpy ( myString , HELLO_STRING ); printLine ( myString ); void printLine (const char * line) if ( line != NULL )printf ( ""%s\n"" , line ); free ( myString );"

list_tokenized_text = tokenizer.texts_to_sequences(source_code)
print(source_code)

data = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_text,
                                                                 maxlen=500,
                                                                 padding='post',
                                                     dtype=object)
data = data.astype(np.int64)

static void goodG2B2() size_t data ; data = 0;if ( STATIC_CONST_FIVE == 5 )data = 20;if ( STATIC_CONST_FIVE == 5 )char * myString ;if ( data > strlen ( HELLO_STRING ) )myString = ( char * ) malloc ( data * sizeof ( char ) ); strcpy ( myString , HELLO_STRING ); printLine ( myString ); void printLine (const char * line) if ( line != NULL )printf ( %s
 , line ); free ( myString );


In [18]:
predicted = model.predict(data)

12/12 [==============================] - 1s 42ms/step


In [19]:
# False - 0, True - 1
pred_test = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]

for col in range(0,len(predicted)):
    for i, row in enumerate(predicted[col]):
        if row[0] >= row[1]:
            pred_test[col].append(0)
        else:
            pred_test[col].append(1)

for col in range(0,len(pred_test)):
    print(pd.value_counts(pred_test[col]))
 

0    380
dtype: int64
0    380
dtype: int64
0    380
dtype: int64
0    380
dtype: int64
0    380
dtype: int64
0    380
dtype: int64
0    380
dtype: int64
0    380
dtype: int64
0    369
1     11
dtype: int64
0    380
dtype: int64
0    380
dtype: int64
0    380
dtype: int64
0    380
dtype: int64
0    380
dtype: int64
0    380
dtype: int64
0    380
dtype: int64
0    380
dtype: int64


In [20]:
column_names = ['CWE114', 'CWE134', 'CWE190', 'CWE196', 'CWE319', 'CWE369', 'CWE398',
               'CWE416', 'CWE427', 'CWE469', 'CWE506', 'CWE605', 'CWE606', 'CWE666',
               'CWE680', 'CWE761', 'CWE789']

indices = [[i for i, val in enumerate(col) if val == 1] for col in pred_test]

indices_list = [i for i, col in enumerate(pred_test) if any(val == 1 for val in col)]
i=[str(index) for index in indices_list]
res_i="".join(i)
res=int(res_i)

print("The vulnerability class detected is: ",column_names[res])


words=[]
for index in indices:
    if len(index) > 0:
        for i in index:
                predicted_word = tokenizer.index_word[i]
                words.append(predicted_word)
                
                
def highlight_words(sentence, words):
    highlighted_sentence = sentence
    for word in words:       
        highlighted_word = f"\033[91m{word}\033[0m"  # Use ANSI escape codes for red color
        highlighted_sentence = highlighted_sentence.replace(word, highlighted_word)
    return highlighted_sentence

sentence = source_code
highlighted_words = words
highlighted_sentence = highlight_words(sentence, highlighted_words)

print()
print("Localized code:")
print()
print(highlighted_sentence)


The vulnerability class detected is:  CWE427

Localized code:

static void goodG2B2() size_t data ; data = 0;if ( STATIC_CONST_FIVE == 5 )data = 20;if ( STATIC_CONST_FIVE == 5 )char * myString ;if ( data > strlen ( HELLO_STRING ) )myString = ( char * ) malloc ( data * sizeof ( char ) ); strcpy ( myString , HELLO_STRING ); printLine ( myString ); void printLine (const char * line) if ( line != NULL )printf ( %s
 , line ); free ( myString );


predicted_token_index = np.argmax(pred_test)

# Get the corresponding word from the vocabulary
predicted_word = tokenizer.index_word[predicted_token_index+1]

# Output the predicted word
print(predicted_word)
print(predicted_token_index)

#Code for vulnerability

b'snd_es1968_pcm(struct es1968 *chip, int device)\n{\n\tstruct snd_pcm *pcm;\n\tint err;\n\n\t/* get DMA buffer */\n\tif ((err = snd_es1968_init_dmabuf(chip)) < 0)\n\t\treturn err;\n\n\t/* set PCMBAR */\n\twave_set_register(chip, 0x01FC, chip->dma.addr >> 12);\n\twave_set_register(chip, 0x01FD, chip->dma.addr >> 12);\n\twave_set_register(chip, 0x01FE, chip->dma.addr >> 12);\n\twave_set_register(chip, 0x01FF, chip->dma.addr >> 12);\n\n\tif ((err = snd_pcm_new(chip->card, ""ESS Maestro"", device,\n\t\t\t       chip->playback_streams,\n\t\t\t       chip->capture_streams, &pcm)) < 0)\n\t\treturn err;\n\n\tpcm->private_data = chip;\n\tpcm->private_free = snd_es1968_pcm_free;\n\n\tsnd_pcm_set_ops(pcm, SNDRV_PCM_STREAM_PLAYBACK, &snd_es1968_playback_ops);\n\tsnd_pcm_set_ops(pcm, SNDRV_PCM_STREAM_CAPTURE, &snd_es1968_capture_ops);\n\n\tpcm->info_flags = 0;\n\n\tstrcpy(pcm->name, ""ESS Maestro"");\n\n\tchip->pcm = pcm;\n\n\treturn 0;\n}'


b'open_uri(const gchar * uri)\n{\n\tint result;\n\tgchar *command;\n\tGError *error = NULL;\n#ifdef WIN32\n\tgchar *browser_command =\n\t\tcfg_get_single_value_as_string_with_default(config, ""Misc"", ""browser-win32"", ""cmd /c start %s"");\n#else\n#ifdef OSX\n\tgchar *browser_command = cfg_get_single_value_as_string_with_default(config, ""Misc"", ""browser-osx"", ""open \'%s\'"");\n#else\n\tgchar *browser_command = cfg_get_single_value_as_string_with_default(config, ""Misc"", ""browser"", ""xdg-open \'%s\'"");\n#endif\n#endif\n\tgchar *escaped_uri = g_strdup(uri);\n\tcommand = g_strdup_printf(browser_command, escaped_uri);\n\tg_free(escaped_uri);\n\tresult = g_spawn_command_line_async(command, &error);\n\tif (error)\n\t{\n\n\t\tgchar *str = g_markup_printf_escaped(""%s: \'%s\': %s"", _(""Failed to execute""), command, error->message);\n\t\tplaylist3_show_error_message(str, ERROR_WARNING);\n\t\tg_free(str);\n\t\tg_error_free(error);\n\t\terror = NULL;\n\t}\n\tg_free(browser_command);\n\tg_free(command);\n}'


b'simple_object_mach_o_write_header (simple_object_write *sobj, int descriptor,\n\t\t\t\t   size_t nsects, const char **errmsg,\n\t\t\t\t   int *err)\n{\n  struct simple_object_mach_o_attributes *attrs =\n    (struct simple_object_mach_o_attributes *) sobj->data;\n  void (*set_32) (unsigned char *, unsigned int);\n  unsigned char hdrbuf[sizeof (struct mach_o_header_64)];\n  unsigned char *hdr;\n  size_t wrsize;\n\n  set_32 = (attrs->is_big_endian\n\t    ? simple_object_set_big_32\n\t    : simple_object_set_little_32);\n\n  memset (hdrbuf, 0, sizeof hdrbuf);\n\n  /* The 32-bit and 64-bit headers start out the same.  */\n\n  hdr = &hdrbuf[0];\n  set_32 (hdr + offsetof (struct mach_o_header_32, magic), attrs->magic);\n  set_32 (hdr + offsetof (struct mach_o_header_32, cputype), attrs->cputype);\n  set_32 (hdr + offsetof (struct mach_o_header_32, cpusubtype),\n\t  attrs->cpusubtype);\n  set_32 (hdr + offsetof (struct mach_o_header_32, filetype), MACH_O_MH_OBJECT);\n  set_32 (hdr + offsetof (struct mach_o_header_32, ncmds), 1);\n  set_32 (hdr + offsetof (struct mach_o_header_32, flags), attrs->flags);\n  if (attrs->magic == MACH_O_MH_MAGIC)\n    {\n      wrsize = sizeof (struct mach_o_header_32);\n      set_32 (hdr + offsetof (struct mach_o_header_32, sizeofcmds),\n\t      (sizeof (struct mach_o_segment_command_32)\n\t       + nsects * sizeof (struct mach_o_section_32)));\n    }\n  else\n    {\n      set_32 (hdr + offsetof (struct mach_o_header_64, sizeofcmds),\n\t      (sizeof (struct mach_o_segment_command_64)\n\t       + nsects * sizeof (struct mach_o_section_64)));\n      set_32 (hdr + offsetof (struct mach_o_header_64, reserved),\n\t      attrs->reserved);\n      wrsize = sizeof (struct mach_o_header_64);\n    }\n\n  return simple_object_internal_write (descriptor, 0, hdrbuf, wrsize,\n\t\t\t\t       errmsg, err);\n}'
